In [ ]:
# https://universe.roboflow.com/selfdriving-car-qtywx/self-driving-cars-lfjou/dataset/2 (dataset link)
# Run this to download the data
# curl -L "https://universe.roboflow.com/ds/s5tubfUJUz?key=dAIVR4GASb" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [1]:
from ultralytics import YOLO

# model = YOLO('yolov8n.pt') # import pre-trained model for object detection

In [ ]:
DATA_PATH = './data.yaml'
IMGSZ = 416

# hyperparameters for tuning
BATCH = 16 # the best size for my GPU
EPOCHS = 30
OPTIMIZER = 'Adam'
LR0 = .001
LRF = .01

results = model.train(
    data = DATA_PATH,
    epochs = EPOCHS,
    batch = BATCH,
    imgsz = IMGSZ,
    optimizer = OPTIMIZER,
    lr0 = .001,
    lrf = .01,
    device = 0
)

In [2]:
model = YOLO('./runs/detect/train2/weights/last.pt')

results = model.train(resume=True)

Ultralytics YOLOv8.0.229 🚀 Python-3.8.10 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce MX150, 2048MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train2/weights/last.pt, data=./data.yaml, epochs=30, time=None, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning /home/kenji/plate_identification/train/labels.cache... 3129 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3129/3129 [00:00<?, ?it/s]
val: Scanning /home/kenji/plate_identification/valid/labels.cache... 758 images, 0 backgrounds, 0 corrupt: 100%|██████████| 758/758 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Closing dataloader mosaic
30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      22/30         1G     0.5334      0.463     0.8932         16        416: 100%|██████████| 196/196 [09:43<00:00,  2.97s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [01:24<00:00,  3.53s/it]

                   all        758        855      0.928       0.85      0.924      0.792


...

      30/30         1G     0.4751     0.3403      0.866         10        416: 100%|██████████| 196/196 [13:02<00:00,  3.99s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [01:16<00:00,  3.18s/it]

                   all        758        855      0.967      0.888      0.946      0.835



9 epochs completed in 2.113 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.0.229 🚀 Python-3.8.10 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce MX150, 2048MiB)
Model summary (fused): 168 layers, 3008378 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [01:03<00:00,  2.64s/it]


                   all        758        855      0.967      0.888      0.946      0.835
           Green Light        758         95      0.952      0.628      0.769      0.477
             Red Light        758         87      0.823      0.563       0.75      0.473
       Speed Limit 100        758         49          1      0.993      0.995      0.909
       Speed Limit 110        758         17      0.999      0.882      0.967      0.907
       Speed Limit 120        758         56      0.982      0.956      0.991      0.921
        Speed Limit 20        758         54      0.988      0.926      0.986      0.874
        Speed Limit 30        758         74      0.972      0.943      0.976      0.914
        Speed Limit 40        758         52      0.976      0.981      0.993      0.877
        Speed Limit 50        758         65      0.979      0.877      0.959      0.873
        Speed Limit 60        758         70      0.966      0.957      0.979      0.904
        Speed Limit 7